In [8]:
!pip install keras-tuner
!pip install tensorflow
!pip install pandas
!pip install scikit-learn


     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     -------------------------------------- 60.6/60.6 kB 645.4 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.2/11.0 MB 3.1 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.0 MB 3.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.0 MB 3.9 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/11.0 MB 4.4 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.0 MB 5.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.0 MB 5.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.0 MB 5.0 MB/s eta 0:00:02
   ------ ------------------------

In [10]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

# Описание данных
[Ссылка на данные](https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/77547/8481168/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1716987718&Signature=hvqq904%2BXKQEtFPkD4OGpnYGbhjwYAj3fHYXLou5DerP4ls%2F2elDkSI7x6LqxValOn8O75%2FcskKAk4J%2Fb9rtKMtNrPjcekJCvtACRLerxgTjbEnsgkeG7TchrxoJCLFjIL343MN3BUyuTp9lT7lD3d6kvQ9nKke3C8gb5ttC1d9d7wVJ61QzTMgsV2NUYm5BtjVmcnQoA6NUZ7s5R5ke%2FYYGfq7xWYMxg%2BO39urSE6zlYmZmr7b93AGQeU2Pyez7NlrDyYQJpI1KATr7JBk0mIQA2uFt9qu0OgLbhjeeD9nO2fJ2nQ%2F%2BL%2BVaUb1dMMzL5ORhtok1jJjjfiEe4rdLpQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dmipt-bio-dl.zip). Для запуска необходимо скачать и скопировать содержимое в папку с data.

Fashion-MNIST – аналог знаменитого датасета MNIST, который состоит не из рукописных символов, а из предметов одежды. Он был создан, чтобы актуализировать учебные задачи в компьютерном зрении и приблизить их к реальности.

Каждое изображение представляет собой набор их 784 пикселей (28x28). Каждый пиксель принимает значение от 0 до 255, обозначая его цвет в черно-белой шкале. Обучающая выборка содержит 786 столбцов, где первый (label) – принадлежность изображения к классу, а последний – его уникальный номер (id). Остальные столбцы – пиксели изображения.
#### Проведите первичный анализ данных, чтобы определить наличие NaN значений.
Тестовая выборка (на основе которой должно быть получено исследование) содержит все те же столбцы, кроме label.

#### Расшифровка классов (label), к которым принадлежат изображения:

0. T-shirt/top – футболка
1. Trouser – брюки
2. Pullover – свитер
3. Dress – платье
4. Coat – пальто
5. Sandal – сандалия
6. Shirt – рубашка
7. Sneaker – кроссовок
8. Bag – сумка
9. Ankle boot – сапог

#### Структура данных:

- fmnist_train.csv – обучающая выборка
- fmnist_test.csv – тестовая выборка (на которой должно быть получено предсказание)
- sample_submission.csv – пример решения

# Загрузка датасетов

In [12]:
train_data = pd.read_csv("data/fmnist_train.csv")
test_data = pd.read_csv("data/fmnist_test.csv")

Проверим на пропуски

In [13]:
# Проверка на наличие NaN значений
print(train_data.isnull().sum().max()) 

1


In [14]:
print("Размерность перед удалением пропусков:", train_data.shape)
print("Размерность перед удалением пропусков:", test_data.shape)

Размерность перед удалением пропусков: (17040, 786)
Размерность перед удалением пропусков: (10000, 785)


Есть пропуски. Посмотрим, сколько их

In [15]:
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)

In [16]:
print("Размерность после удаления пропусков:", train_data.shape)
print("Размерность после удаления пропусков:", test_data.shape)

Размерность после удаления пропусков: (17039, 786)
Размерность после удаления пропусков: (10000, 785)


Пропуск был всего один, поэтому ничего страшного в том, чтобы его удалить.

In [17]:

# Разделение данных на признаки и метки
X_train = train_data.drop(['label', 'Id'], axis=1).values
y_train = train_data['label'].values

X_test = test_data.drop(['Id'], axis=1).values

# Нормализация
X_train = X_train.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

# Изменение размера данных
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Преобразование меток в категории
y_train = to_categorical(y_train, num_classes=10)



# Построение модели
Модель основана на типичной архитектуре **сверточной нейронной сети** (Convolutional Neural Network, CNN), которая широко используется для задач обработки изображений. **Сверточные нейронные сети** хорошо подходят для распознавания визуальных паттернов непосредственно из пикселей изображения с минимальной предварительной обработкой. Они могут автоматически изучать иерархии признаков, что делает их особенно эффективными для задач таких, как классификация изображений.


### Conv2D слои:
Эти слои выполняют свертку, используя учебные фильтры для выделения важных признаков из входных изображений. Выбор размера фильтров (kernel size) и количества фильтров (filters) влияет на способность модели извлекать информативные признаки.
Например, первый слой может иметь фильтры размером 3x3 или 5x5, которые изучают маленькие детали в изображении. Увеличив число фильтров, можно позволить сети изучить больше различных аспектов входных данных.
### MaxPooling2D слои:
Эти слои уменьшают размерность пространственных данных, сохраняя при этом наиболее значимую информацию (например, выделяют самые яркие или самые темные участки). Это помогает сократить объем вычислений и контролировать переобучение.
### Dropout слои:
Эти слои случайно исключают некоторые узлы во время обучения, что помогает предотвратить переобучение, увеличивая способность модели к обобщению.
### Flatten слой:
Этот слой преобразует многомерную карту признаков, полученную после всех сверток и пулингов, в одномерный вектор. Это необходимо для того, чтобы данный вектор можно было подать на полносвязные (Dense) слои.
### Dense слои:
Эти слои представляют собой полносвязные слои, которые выполняют классификацию на основе выделенных признаков. Выбор количества нейронов и функций активации в этих слоях способствует окончательному принятию решений.
Слой с активацией 'relu' используется для добавления нелинейности в модель, а последний слой с активацией 'softmax' используется для классификации на несколько классов, где он выводит вероятности принадлежности к каждому из классов.

In [21]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.45))
model.add(Dense(10, activation='softmax'))

# Компиляция модели
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10


D:\git\github\MIPT\DL\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.5959 - loss: 1.1477 - val_accuracy: 0.8063 - val_loss: 0.5177
Epoch 2/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.8113 - loss: 0.5327 - val_accuracy: 0.8454 - val_loss: 0.4187
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8388 - loss: 0.4428 - val_accuracy: 0.8556 - val_loss: 0.3983
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.8570 - loss: 0.4019 - val_accuracy: 0.8636 - val_loss: 0.3678
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8667 - loss: 0.3734 - val_accuracy: 0.8721 - val_loss: 0.3447
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8829 - loss: 0.3371 - val_accuracy: 0.8727 - val_loss: 0.3410
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8854 - loss: 0.3122 - val_accuracy: 0.8803 - val_loss: 0.3233
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.8906 - loss: 0.2918 - val_accuracy: 0.881

# Предсказание на тестовой выборке

In [22]:
predictions = model.predict(X_test)
predictions = np.argmax(predictions, axis=1)

submission_1 = pd.DataFrame({'id': test_data['Id'], 'label': predictions})
submission_1.to_csv('data/outcome.csv', index=False)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
